In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import os
from keras import layers
tf.test.is_gpu_available()

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.python.training'

In [6]:
df= pd.read_csv("../../scene-classification/train.csv")

df = df.replace(0,'buildings').replace(1,'forest').replace(2,'glacier').replace(3,'mountain').replace(4,'sea').replace(5,'street')

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import preprocessing

datagen = ImageDataGenerator(rescale=1./255,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip=True,
                           preprocessing_function= preprocess_input,  
                           validation_split=0.25)
train_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="../../scene-classification/train",
                        x_col="image_name",
                        y_col="label",
                        subset="training",
                        batch_size=16,
                        class_mode="categorical",
                        target_size=(150,150))
valid_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="../../scene-classification/train",
                        x_col="image_name",
                        y_col="label",
                        subset="validation", 
                        batch_size=10,
                        class_mode="categorical",
                        target_size=(150,150))

Found 12776 validated image filenames belonging to 6 classes.
Found 4258 validated image filenames belonging to 6 classes.


In [8]:
from keras.applications.vgg16 import VGG16
vgg_conv = VGG16(weights = 'imagenet',include_top = False, input_shape = (150,150,3))

In [9]:
import tensorflow as tf
print(tf.__version__)

for layer in vgg_conv.layers[8:]:
    layer.trainable = True

for layer in vgg_conv.layers:
    print(layer,layer.trainable)
    

1.2.0
<keras.engine.input_layer.InputLayer object at 0x7f02d96e2b00> False
<keras.layers.convolutional.Conv2D object at 0x7f0240f346a0> True
<keras.layers.convolutional.Conv2D object at 0x7f028876b710> True
<keras.layers.pooling.MaxPooling2D object at 0x7f0240c3c9e8> True
<keras.layers.convolutional.Conv2D object at 0x7f0240c56358> True
<keras.layers.convolutional.Conv2D object at 0x7f0240bee5f8> True
<keras.layers.pooling.MaxPooling2D object at 0x7f0240c05d30> True
<keras.layers.convolutional.Conv2D object at 0x7f0240c17630> True
<keras.layers.convolutional.Conv2D object at 0x7f0240bb2748> True
<keras.layers.convolutional.Conv2D object at 0x7f0240bc9fd0> True
<keras.layers.pooling.MaxPooling2D object at 0x7f0240b76908> True
<keras.layers.convolutional.Conv2D object at 0x7f0240b76c88> True
<keras.layers.convolutional.Conv2D object at 0x7f0240b26320> True
<keras.layers.convolutional.Conv2D object at 0x7f0240b3bc50> True
<keras.layers.pooling.MaxPooling2D object at 0x7f0240aea4e0> True
<

In [10]:
# from keras import models,layers,optimizers
# model_vgg = models.Sequential()
# # model_vgg = vgg_conv
# model_vgg.add(vgg_conv)
# model_vgg.add(layers.Flatten())
# model_vgg.add(layers.Dense(1024,  activation='relu'))
# model_vgg.add(layers.Dropout(0.4))
# model_vgg.add(layers.Dense(6,activation = 'softmax'))

# model_vgg.summary()

In [11]:
from keras import models,layers,optimizers
from keras.models import Model
x = vgg_conv.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1024, activation="relu")(x)
predictions = layers.Dense(6, activation="softmax")(x)
model_vgg = Model(input = vgg_conv.input, output = predictions)


TypeError: softmax() got an unexpected keyword argument 'axis'

In [9]:
model_vgg.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4),metrics=['acc'])

W0822 09:33:58.730444 140126876735296 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
history = model_vgg.fit_generator(train_generator, 
                                  steps_per_epoch = train_generator.samples/train_generator.batch_size,
                                  epochs = 10, validation_data=valid_generator, validation_steps = 80, use_multiprocessing = True)


W0822 09:34:04.859469 140126876735296 deprecation.py:323] From /home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
 44/798 [>.............................] - ETA: 1:27:18 - loss: 12.1018 - acc: 0.2131

Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = ge

KeyboardInterrupt: 

In [ ]:
from keras.callbacks import ModelCheckpoint
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

history =model_vgg.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator, callbacks=[checkpointer],
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,verbose=1)

In [1]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

InvalidArgumentError: Cannot assign a device for operation MatMul: node MatMul (defined at <ipython-input-1-3a5be606bcc9>:5) was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:XLA_CPU:0, /job:localhost/replica:0/task:0/device:XLA_GPU:0 ]. Make sure the device specification refers to a valid device.
	 [[MatMul]]

Errors may have originated from an input operation.
Input Source operations connected to node MatMul:
 b (defined at <ipython-input-1-3a5be606bcc9>:4)	
 a (defined at <ipython-input-1-3a5be606bcc9>:3)

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2885001066981296235
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7242989902859744675
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11682321363820510323
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/test_project_keras_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

Default GPU Device: /gpu:0


In [2]:
tf.test.is_gpu_available()

False